<a href="https://colab.research.google.com/github/daycardoso/bert-vs-modernbert-valueeval24/blob/main/valores_modern_bert_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inicialização

In [1]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import userdata
import os
os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')
os.environ['WANDB_API_KEY'] = userdata.get('WANDB_API_KEY')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import json
import os # Adicionado para construir os caminhos

# Carregar os Dados em Ingles
home = "/content/drive/MyDrive/Mestrado/DetectionOfHumanValuesInTexts/Colab_Experimentos/"

folder_treino = home + "training-english/"
folder_validacao = home + "validation-english/"
folder_teste = home + "test-english/"

# Json value-categories - Carregue o JSON uma única vez
caminho_json_valores = os.path.join(home, "value-categories.json")
with open(caminho_json_valores, 'r') as f:
    categorias_valores = json.load(f)

# Json value-categories
categorias_valores = json.load(open(home + "value-categories.json"))


# Aplicando o pre-processamento de adição de contexto do Hierocles of Alexandria at Touché

# Treinamento dos modelos para cada direction

In [3]:
!pip install "numpy<2.0"

In [7]:
# -*- coding: utf-8 -*-
"""
Refatoração do script de treinamento para usar ModernBertForSequenceClassification.
"""

import os
import random
import numpy as np
import pandas as pd
import torch
import datasets
import wandb
from scipy.special import expit as sigmoid
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score
from tqdm.auto import tqdm
from transformers import (
    AutoTokenizer,
    AutoConfig,
    ModernBertForSequenceClassification, # <-- Importação principal alterada
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)

# --- 1. Inicialização e Configuração Global ---

# Inicializa Weights & Biases para monitoramento
try:
    wandb.init(
        project='touche_multi_head',
        name='modernbert_seq_class_19_values_context_refactored'
    )
except Exception as e:
    print(f"W&B initialization failed: {e}. Running without logging.")

# Definição dos valores (labels) e tokens especiais
VALORES = [
    'Self-direction: thought', 'Self-direction: action', 'Stimulation', 'Hedonism',
    'Achievement', 'Power: dominance', 'Power: resources', 'Face',
    'Security: personal', 'Security: societal', 'Tradition', 'Conformity: rules',
    'Conformity: interpersonal', 'Humility', 'Benevolence: caring',
    'Benevolence: dependability', 'Universalism: concern', 'Universalism: nature',
    'Universalism: tolerance'
]
SPECIAL_TOKENS = ['<NONE>'] + [f'<{valor}>' for valor in VALORES]
NUM_LABELS = len(VALORES)
ID2LABEL = {i: l for i, l in enumerate(VALORES)}
LABEL2ID = {l: i for i, l in enumerate(VALORES)}

PRETRAINED_MODEL = 'answerdotai/ModernBERT-base'
MAX_LENGTH = 512

# --- 2. Preparação do Tokenizador ---

tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL)
tokenizer.add_special_tokens({'additional_special_tokens': SPECIAL_TOKENS + ['</s>']})

# --- 3. Funções de Processamento de Dados ---

def preprocess_function(examples):
    """Tokeniza os textos do dataset."""
    return tokenizer(examples['Text'], padding='max_length', truncation=True, max_length=MAX_LENGTH)

def _add_context_to_text(data_df):
    """
    Adiciona sentenças anteriores e seus rótulos como contexto.
    Esta é uma função auxiliar para manter a lógica de negócio isolada.
    """
    data_df = data_df.sort_values(['Text-ID', 'Sentence-ID']).reset_index(drop=True)
    new_texts = []

    # Armazena um exemplo aleatório para verificação
    random_idx = random.randint(0, len(data_df) - 1)
    sample_info = {}

    for i in tqdm(range(len(data_df)), desc="Adicionando contexto"):
        current_text = data_df['Text'][i]
        context_parts = []

        # Itera sobre as duas sentenças anteriores, se existirem
        for j in range(1, 3):
            prev_idx = i - j
            # Garante que o índice é válido e pertence ao mesmo Text-ID
            if prev_idx >= 0 and data_df['Text-ID'][prev_idx] == data_df['Text-ID'][i]:
                prev_text = data_df['Text'][prev_idx]
                prev_labels = data_df['labels'][prev_idx]

                # Obtém os nomes dos rótulos ativos
                active_labels = [ID2LABEL[k] for k, label in enumerate(prev_labels) if label == 1]

                # Formata o texto anterior com seus rótulos
                if active_labels:
                    context_text = prev_text + ' ' + ' '.join([f'<{l}>' for l in active_labels])
                else:
                    context_text = prev_text + ' <NONE>'
                context_parts.insert(0, context_text)

        # Junta o contexto com o texto atual
        if context_parts:
            processed_text = ' </s> '.join(context_parts) + ' </s> ' + current_text
        else:
            processed_text = current_text

        new_texts.append(processed_text)

        # Salva o exemplo aleatório para inspeção
        if i == random_idx:
            sample_info = {
                'Text-ID': data_df['Text-ID'][i],
                'Sentence-ID': data_df['Sentence-ID'][i],
                'original_text': current_text,
                'processed_text': processed_text,
                'labels': data_df['labels'][i]
            }

    data_df['Text'] = new_texts
    return data_df, sample_info


def load_and_process_dataset(directory, tokenizer_instance):
    """
    Carrega, mescla, processa e tokeniza o dataset de um diretório,
    com correção de tipo de dados para o merge.
    """
    sentences_file_path = os.path.join(directory, 'final_sentences.tsv')
    labels_file_path = os.path.join(directory, 'final_labels.tsv')

    if not os.path.exists(sentences_file_path) or not os.path.exists(labels_file_path):
        raise FileNotFoundError(f"Arquivos de dataset não encontrados em {directory}")

    # --- CORREÇÃO APLICADA AQUI ---
    # 1. Definir os tipos de dados para as colunas-chave para garantir consistência.
    key_column_types = {'Text-ID': str, 'Sentence-ID': str}

    # 2. Carregar os dados especificando os tipos.
    #    'usecols' torna o carregamento mais seguro e eficiente em memória.
    #    Assumimos que o arquivo de sentenças contém a coluna 'Text'.
    try:
        data_df = pd.read_csv(
            sentences_file_path,
            sep='\t',
            dtype=key_column_types,
            usecols=['Text-ID', 'Sentence-ID', 'Text']
        )
    except ValueError as e:
        raise ValueError(f"Verifique se 'final_sentences.tsv' contém as colunas 'Text-ID', 'Sentence-ID' e 'Text'. Erro: {e}")

    labels_df = pd.read_csv(labels_file_path, sep='\t', dtype=key_column_types)
    # ---------------------------------

    # Mescla os dataframes. Agora os tipos das chaves são idênticos.
    # O 'how="inner"' (padrão) garante que apenas linhas com IDs em ambos os arquivos sejam mantidas.
    merged_df = pd.merge(data_df, labels_df, on=['Text-ID', 'Sentence-ID'])

    # Converte os labels para o formato multi-hot e tipo correto para o Trainer
    labels_matrix = merged_df[VALORES].values.astype(np.float32)
    # Convert the numpy array directly into a list of numpy arrays or keep it as a numpy array
    # before creating the dataset. Converting to a list of Tensors earlier caused the Arrow error.
    merged_df['labels'] = [row.astype(np.float32) for row in labels_matrix]


    # Adiciona o contexto (lógica de negócio principal)
    processed_df, sample = _add_context_to_text(merged_df)

    # Exibe um exemplo aleatório para validação
    print("\n" + "="*35)
    print(f"=== Exemplo de Texto Pré-processado ({os.path.basename(directory)}) ===")
    print(f"Text-ID: {sample['Text-ID']}")
    print(f"Sentence-ID: {sample['Sentence-ID']}")
    print(f"Texto Original: {sample['original_text']}")
    print(f"Texto com Contexto: {sample['processed_text']}")
    # Ensure sample['labels'] is a numpy array for this operation
    active_labels = [ID2LABEL[i] for i, label in enumerate(sample['labels'].tolist()) if label == 1] # Convert back to list for printing
    print(f"Rótulos: {active_labels if active_labels else 'Nenhum'}")
    print("="*35 + "\n")

    # Cria o dataset Hugging Face e tokeniza
    dataset = datasets.Dataset.from_pandas(processed_df)
    dataset = dataset.map(preprocess_function, batched=True, load_from_cache_file=False)

    # Remove colunas desnecessárias e formata para o Trainer
    valid_cols = ['input_ids', 'attention_mask', 'labels']
    dataset = dataset.remove_columns([c for c in dataset.column_names if c not in valid_cols])
    dataset.set_format("torch")

    return dataset

# --- 4. Função de Métricas ---

def compute_metrics(p):
    """Calcula e retorna um dicionário de métricas de classificação multi-label."""
    logits = p.predictions
    true_labels = p.label_ids

    # Aplica sigmoide e define o threshold para obter as predições
    probs = sigmoid(logits)
    preds = (probs > 0.5).astype(int)

    # Calcula as métricas
    f1_macro = f1_score(true_labels, preds, average='macro', zero_division=0)
    f1_micro = f1_score(true_labels, preds, average='micro', zero_division=0)
    precision_macro = precision_score(true_labels, preds, average='macro', zero_division=0)
    recall_macro = recall_score(true_labels, preds, average='macro', zero_division=0)
    subset_accuracy = (true_labels == preds).all(axis=1).mean()

    # Calcula ROC-AUC por label e depois a média
    auc_scores = []
    for i in range(NUM_LABELS):
        if len(np.unique(true_labels[:, i])) > 1: # ROC AUC requer ambas as classes
            auc_scores.append(roc_auc_score(true_labels[:, i], probs[:, i]))
    roc_auc = np.mean(auc_scores) if auc_scores else float('nan')

    return {
        'subset_accuracy': subset_accuracy,
        'f1_macro': f1_macro,
        'f1_micro': f1_micro,
        'precision_macro': precision_macro,
        'recall_macro': recall_macro,
        'roc_auc': roc_auc
    }

# --- 5. Lógica Principal de Treinamento ---

def main():
    """Função principal que executa todo o pipeline."""

    print('Carregando e processando datasets...')
    train_ds = load_and_process_dataset(folder_treino, tokenizer)
    val_ds = load_and_process_dataset(folder_validacao, tokenizer)
    print('Datasets prontos.')

    # Configuração de treinamento
    training_args = TrainingArguments(
        output_dir='modernbert-base-seq-class-values-context',
        report_to='wandb',
        eval_strategy='steps',
        eval_steps=767,
        save_strategy='steps',
        save_steps=767,
        save_total_limit=2,
        learning_rate=5e-6,
        adam_epsilon=1e-8,
        num_train_epochs=33,
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        gradient_accumulation_steps=8,
        weight_decay=0.01,
        load_best_model_at_end=True,
        metric_for_best_model='f1_macro',
        greater_is_better=True,
        fp16=True,
        lr_scheduler_type='linear',
        warmup_ratio=0.01,
        seed=2025,
        overwrite_output_dir=True,
        push_to_hub=True,
        hub_model_id='DayCardoso/modernbert-base-multi-head-values-context',
    )

    print('Inicializando modelo...')
    config = AutoConfig.from_pretrained(
        PRETRAINED_MODEL,
        num_labels=NUM_LABELS,
        id2label=ID2LABEL,
        label2id=LABEL2ID,
        problem_type='multi_label_classification',
    )

    # Instanciação do modelo usando a classe da biblioteca
    model = ModernBertForSequenceClassification.from_pretrained(
        PRETRAINED_MODEL,
        config=config,
    )

    # Redimensiona os embeddings para incluir os novos tokens especiais
    model.resize_token_embeddings(len(tokenizer))

    # Inicializa o Trainer
    from transformers import DataCollatorWithPadding

    # O DataCollator cuida de agrupar e preencher os dados de forma inteligente
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        compute_metrics=compute_metrics,
        data_collator=data_collator, # <-- Argumento novo e recomendado
        callbacks=[EarlyStoppingCallback(early_stopping_patience=9)],
    )

    print('Iniciando treinamento...')
    train_res = trainer.train()

    # Salva e loga as métricas finais
    metrics = train_res.metrics
    metrics['train_samples'] = len(train_ds)
    trainer.log_metrics('train', metrics)
    trainer.save_model()
    trainer.save_state()
    trainer.save_metrics('train', metrics)

    print('\nAvaliação final no dataset de validação...')
    eval_res = trainer.evaluate()
    eval_res['eval_samples'] = len(val_ds)
    trainer.log_metrics('eval', eval_res)
    trainer.save_metrics('eval', eval_res)
    print('Treinamento concluído! Métricas de avaliação:', eval_res)

    wandb.finish()

if __name__ == '__main__':
    main()

eval/f1_macro,▁
eval/f1_micro,▁
eval/loss,▁
eval/precision_macro,▁
eval/recall_macro,▁
eval/roc_auc,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
eval/subset_accuracy,▁
train/epoch,▁


Carregando e processando datasets...


Adicionando contexto:   0%|          | 0/24534 [00:00<?, ?it/s]


=== Exemplo de Texto Pré-processado () ===
Text-ID: IT_M_005
Sentence-ID: 8
Texto Original: The road to full equality is still a long way off.
Texto com Contexto: There are still numerous difficulties for those who decide to go into freelance practice. <Universalism: concern> </s> From treatment at the pension level compared to employment to discrimination leading to exclusion from incentives and benefits granted to other economic entities. <Conformity: rules> </s> The road to full equality is still a long way off.
Rótulos: ['Achievement']



Map:   0%|          | 0/24534 [00:00<?, ? examples/s]

...

Iniciando treinamento...


Step,Training Loss,Validation Loss,Subset Accuracy,F1 Macro,F1 Micro,Precision Macro,Recall Macro,Roc Auc
767,2.545100,0.201224,0.002716,0.002297,0.004995,0.071770,0.001170,0.653061
1534,1.507500,0.183802,0.076800,0.056927,0.131853,0.232956,0.035314,0.743661
2301,1.438200,0.178097,0.143721,0.131816,0.228098,0.353364,0.089111,0.779157
3068,1.385800,0.170959,0.168045,0.158214,0.261538,0.433774,0.109087,0.796180
3835,1.315700,0.168094,0.182245,0.178662,0.279626,0.496668,0.126727,0.805761
4602,1.291000,0.162240,0.222867,0.211544,0.329104,0.630203,0.152305,0.819525
5369,1.238800,0.161357,0.202618,0.220097,0.308180,0.614294,0.153591,0.822242
6136,1.199300,0.158333,0.244475,0.245391,0.355416,0.595627,0.178329,0.829054
6903,1.141500,0.160782,0.279294,0.288311,0.393366,0.561370,0.221953,0.828819
7670,1.122100,0.159541,0.238424,0.252289,0.353342,0.598198,0.176072,0.834184


***** train metrics *****
  epoch                    =     17.5002
  total_flos               = 136272631GF
  train_loss               =      0.7183
  train_runtime            =  5:42:23.00
  train_samples            =       24534
  train_samples_per_second =      39.411
  train_steps_per_second   =       2.464

Avaliação final no dataset de validação...


***** eval metrics *****
  epoch                   =    17.5002
  eval_f1_macro           =     0.3176
  eval_f1_micro           =     0.3956
  eval_loss               =     0.2275
  eval_precision_macro    =     0.4188
  eval_recall_macro       =     0.2643
  eval_roc_auc            =     0.8003
  eval_runtime            = 0:01:59.28
  eval_samples            =       8099
  eval_samples_per_second =     67.899
  eval_steps_per_second   =     33.954
  eval_subset_accuracy    =     0.2824
Treinamento concluído! Métricas de avaliação: {'eval_loss': 0.22748833894729614, 'eval_subset_accuracy': 0.2823805408075071, 'eval_f1_macro': 0.3176270171277935, 'eval_f1_micro': 0.39561431796194774, 'eval_precision_macro': 0.41876004325734956, 'eval_recall_macro': 0.26431465786370256, 'eval_roc_auc': 0.8003224032197798, 'eval_runtime': 119.2801, 'eval_samples_per_second': 67.899, 'eval_steps_per_second': 33.954, 'epoch': 17.500203798809814, 'eval_samples': 8099}


eval/f1_macro,▁▂▄▄▅▆▆▆▇▇▇▇▇███████████████████████
eval/f1_micro,▁▃▅▆▆▇▆▇█▇██████████████████████████
eval/loss,▃▂▂▂▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▃▃▃▄▄▄▄▅▅▆▆▆▇▇▇█▄
eval/precision_macro,▁▃▅▆▆███▇█▇▇▇▇▇▆▆▆▆▆▆▆▆▆▅▅▅▅▆▅▅▆▆▆▅▅
eval/recall_macro,▁▂▃▄▄▅▅▆▇▆▇▇▇▇▇▇█▇██████████████████
eval/roc_auc,▁▅▆▇▇▇██████████▇▇▇▇▇▇▇▇▇▇▇▇▇▆▆▆▆▆▆▇
eval/runtime,▂▁▁▁▂▂▆▅▅▅▄▅▄▅▄▅▅▆▆▅▆█▅▃▃▄▃▃▃▇▄▆▄▅▃▂
eval/samples_per_second,▇███▇▇▃▄▄▄▄▄▅▄▅▄▄▃▃▄▃▁▄▆▆▅▆▆▆▂▅▃▅▄▆▇
eval/steps_per_second,▇███▇▇▃▄▄▄▄▄▅▄▅▄▄▃▃▄▃▁▄▆▆▅▆▆▆▂▅▃▅▄▆▇
eval/subset_accuracy,▁▃▄▅▅▆▆▇█▇█▇█▇██████████████████████
train/epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████


In [ ]:
# --- 5. Lógica Principal de Treinamento ---

def main():
    """Função principal que executa todo o pipeline."""

    print('Carregando e processando datasets...')
    train_ds = load_and_process_dataset(folder_treino, tokenizer)
    val_ds = load_and_process_dataset(folder_validacao, tokenizer)
    print('Datasets prontos.')

    # Configuração de treinamento
    training_args = TrainingArguments(
        output_dir='modernbert-base-seq-class-values-context',
        report_to='wandb',
        eval_strategy='e
        save_total_limit=2,
        learning_rate=5e-6,
        adam_epsilon=1e-8,
        num_train_epochs=33,
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        gradient_accumulation_steps=8,
        weight_decay=0.01,
        load_best_model_at_end=True,
        metric_for_best_model='f1_macro',
        greater_is_better=True,
        fp16=True,
        lr_scheduler_type='linear',
        warmup_ratio=0.01,
        seed=2025,
        overwrite_output_dir=True,
        push_to_hub=True,
        hub_model_id='DayCardoso/modernbert-base-multi-head-values-context',
    )

    print('Inicializando modelo...')
    config = AutoConfig.from_pretrained(
        PRETRAINED_MODEL,
        num_labels=NUM_LABELS,
        id2label=ID2LABEL,
        label2id=LABEL2ID,
        problem_type='multi_label_classification',
    )

    # Instanciação do modelo usando a classe da biblioteca
    model = ModernBertForSequenceClassification.from_pretrained(
        PRETRAINED_MODEL,
        config=config,
    )

    # Redimensiona os embeddings para incluir os novos tokens especiais
    model.resize_token_embeddings(len(tokenizer))

    # Inicializa o Trainer
    from transformers import DataCollatorWithPadding

    # O DataCollator cuida de agrupar e preencher os dados de forma inteligente
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        compute_metrics=compute_metrics,
        data_collator=data_collator, # <-- Argumento novo e recomendado
        callbacks=[EarlyStoppingCallback(early_stopping_patience=9)],
    )

    print('Iniciando treinamento...')
    train_res = trainer.train()

    # Salva e loga as métricas finais
    metrics = train_res.metrics
    metrics['train_samples'] = len(train_ds)
    trainer.log_metrics('train', metrics)
    trainer.save_model()
    trainer.save_state()
    trainer.save_metrics('train', metrics)

    print('\nAvaliação final no dataset de validação...')
    eval_res = trainer.evaluate()
    eval_res['eval_samples'] = len(val_ds)
    trainer.log_metrics('eval', eval_res)
    trainer.save_metrics('eval', eval_res)
    print('Treinamento concluído! Métricas de avaliação:', eval_res)

    wandb.finish()

if __name__ == '__main__':
    main()